In [88]:
from __future__ import print_function
import torch
import numpy as np

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU



In [ ]:
#Create a randon tensor

#tensor size
m,n,t= 6,6,6
# create sparse tensor X with ones at idx 
idx = torch.LongTensor([[0, 1, 1],
                          [2, 0, 2],
                           [1,1,2]]) #idx for 
value = torch.ones(3)
X=torch.sparse.FloatTensor(idx, value, torch.Size([m,n,t])).to_dense()

In [89]:
#SVD 
def svd(X):
    r=3 
    #Mode-1 folding
    X1= X.reshape(m,n*t)
    U,S,V= torch.svd(X1,some=True, compute_uv=True) 
    U1=U[:,:r]
    #Mode-2 folding
    X2= X.reshape(m*t,n) 
    U,S,V= torch.svd(X2,some=True, compute_uv=True) 
    V1=V[:,:r]
    return U1,V1
U1,V1= svd(X)
U1,V1


(tensor([[0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 1.],
         [0., 0., 0.]]), tensor([[-0., -0.,  0.],
         [-1., -0.,  0.],
         [-0., -1.,  0.],
         [-0., -0.,  0.],
         [-0., -0.,  1.],
         [-0., -0.,  0.]]))

In [92]:
#Update S based on U1, V1
def update_s(U, X, V):
    new_s = torch.empty((r,r,t))
    for i in range(t): 
        m1 = torch.mm(torch.t(U),X[i]) #torch.t = transpose
        m2 = torch.mm(m1,V)
        new_s[:,:,i]=m2
    return new_s
S=update_s(U1,X,V1)

In [103]:
#Compute Z:
def update_z(U,V,S):
    new_z = torch.empty((m,n,t))
    for i in range(t): 
        S_t=S[:,:,i]
        m1 = torch.mm(U,S_t)
        m2 = torch.mm(m1,torch.t(V))
        new_z[:,:,i]=m2
    return new_z
Z=update_z(U1,V1,S)



(array([[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.]], dtype=float32),
 array([1.4142135, 1.       , 0.       ], dtype=float32),
 array([[ 0.        ,  0.70710677,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.70710677],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.     

In [49]:
# useful website for operations: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/
X_5= torch.cat((X,X,X), dim=2) # Concatenate X's in the 0 dimension

X.shape, X_5.shape

(torch.Size([6, 6, 6]), torch.Size([6, 6, 18]))